### Name: Mansi Mrugen Shah
### NetID: ws2865

### Build a word level sequence to sequence model for English to Marathi.
Word level models can learn syntactically and grammatically correct sentences and are more robust than character-based ones.Word level models are more memory intensive than character level models and cannot learnt out of vocabulary words. But, requires lesser training than character level models.

In this model, I have used loss function as categorical crossentropy, adam optimizer and output function as softmax. I have achieved 61.5% validation accuracy.

#### Import the required libraries 

In [1]:
%tensorflow_version 1.x
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

TensorFlow 1.x selected.


### Data loading and cleaning: remove punctuations, digits and convert to lower case

In [0]:
lines= pd.read_table('/content/mar.txt', names=['eng', 'mar', 'na'])
lines = lines.drop(columns = ['na'])

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [0]:

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of marathi
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [9]:
# Max Length of source sequence
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)
max_length_src

35

In [10]:
# Max Length of target sequence
length_list=[]
for l in lines.mar:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)
max_length_tar

37

In [11]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5762, 13566)

In [12]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens
num_encoder_tokens += 1 
num_encoder_tokens

5763

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])


In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [15]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
278,i can win,START_ मी जिंकू शकतो _END
4587,thats my dress,START_ तो माझा ड्रेस आहे _END
12332,where did you study,START_ कुठे अभ्यास केलास _END
13210,i was the only woman,START_ मी एकटीच महिला होती _END
21310,tom sold his gun to mary,START_ टॉमने त्याची बंदूक मेरीला विकून टाकली _END
16159,what can you teach me,START_ तुम्ही मला काय शिकवू शकता _END
31677,he took a room at the yaesu hotel,START_ त्याने याएसु हॉटेलात एक खोली घेतली _END
36367,you should have left half an hour earlier,START_ तुम्ही अर्धा तास अगोदर निघायला हवं होतं...
14310,which way did tom go,START_ टॉम कोणत्या दिशेने गेला _END
34096,tom answered all of marys questions,START_ टॉमने मेरीच्या सगळ्या प्रश्नांची उत्तरं...


### Train and validation split

In [16]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)
X_train.shape, X_test.shape

((36761,), (1935,))

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:

latent_dim = 256

In [19]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Create the model

In [0]:

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [22]:

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 150
print(val_samples//batch_size)

15


In [23]:
history = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/150
287/287 [==============================] - 87s 302ms/step - loss: 0.8793 - acc: 0.2113 - val_loss: 0.7966 - val_acc: 0.2475
Epoch 2/150
287/287 [==============================] - 82s 285ms/step - loss: 0.7446 - acc: 0.2791 - val_loss: 0.6674 - val_acc: 0.3105
Epoch 3/150
287/287 [==============================] - 82s 284ms/step - loss: 0.6509 - acc: 0.3477 - val_loss: 0.6032 - val_acc: 0.3708
Epoch 4/150
287/287 [==============================] - 82s 285ms/step - loss: 0.5764 - acc: 0.4026 - val_loss: 0.5503 - val_acc: 0.4143
Epoch 5/150
287/287 [==============================] - 81s 284ms/step - loss: 0.5143 - acc: 0.4526 - val_loss: 0.5160 - val_acc: 0.4432
Epoch 6/150
287/287 [==============================] - 81s 283ms/step - loss: 0.4627 - acc: 0.4957 - val_loss: 0.4830 - val_acc: 0.4748
Epoch 7/150
287/287 [==============================] - 81s 281ms/step - loss: 0.4202 - acc: 0.5349 - val_loss: 0.4577 - val_acc: 0.5036
Epoch 8/150
287/287 [===========================

### Maximum Validation Accuracy

In [25]:
print("Maximum accuracy of validation: ", max(history.history['val_acc']))

Maximum accuracy of validation:  0.6153016


### Predictions

In [0]:

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [0]:

val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [32]:

for dd in range(0,50):
  k+=1
  (input_seq, actual_output), _ = next(val_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input English sentence:', X_test[k:k+1].values[0])
  print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
  print('Predicted Marathi Translation:', decoded_sentence[:-4])
  print('\n')

Input English sentence: i had fun here
Actual Marathi Translation:  मला इथे मजा आली 
Predicted Marathi Translation:  मी इथे मजा आली 


Input English sentence: this book has been translated into more than fifty languages
Actual Marathi Translation:  या पुस्तकाचा पन्नासपेक्षा जास्त भाषांमध्ये अनुवाद केला गेला आहे 
Predicted Marathi Translation:  या पुस्तकात इंग्रजीत केलंस 


Input English sentence: i have seen that girl before
Actual Marathi Translation:  मी त्या मुलीला याआधी पाहिलं आहे 
Predicted Marathi Translation:  मी त्या जे त्या मला विकत बघितली 


Input English sentence: tom and i danced with each other
Actual Marathi Translation:  मी आणि टॉम एकमेकांसोबत नाचलो 
Predicted Marathi Translation:  टॉम आणि मी काल एकमेकांना यायला सांगितलं 


Input English sentence: dont tell anyone this
Actual Marathi Translation:  असं कोणाला सांगू नका 
Predicted Marathi Translation:  असं कोणाला सांगू नकोस 


Input English sentence: hes already left
Actual Marathi Translation:  ते यापुर्वीच निघालेयत 
Pred